In [2]:
import datetime as datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import pyspark

plt.style.use('default') # Make the graphs a bit prettier
plt.rcParams['figure.figsize'] = (15, 5)
%matplotlib inline

/usr/lib/python2.7/dist-packages/matplotlib/__init__.py:874: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [3]:
train = pd.read_csv('data/trips_train_with_stationweather.csv', low_memory=False)
train_filtered = pd.read_csv('data/trips_train_with_stationweather_filtered.csv', low_memory=False)
test = pd.read_csv('data/trips_test_with_stationweather.csv', low_memory=False)

In [4]:
print train.shape
train.head(2)


(549961, 17)


,duration,season,start_station_id,end_station_id,start_hour,day_of_week,subscription_type,mean_temperature_f,mean_dew_point_f,mean_humidity,mean_sea_level_pressure_inches,mean_visibility_miles,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,events,wind_dir_degrees
0,396,summer,50,60,8,3,Subscriber,72.0,55.0,57.0,30.01,10.0,6.0,29.0,0,Clear,272.0
1,981,summer,50,82,9,3,Subscriber,72.0,55.0,57.0,30.01,10.0,6.0,29.0,0,Clear,272.0


In [5]:
test.head(2)

,trip_id,season,start_station_id,end_station_id,start_hour,day_of_week,subscription_type,mean_temperature_f,mean_dew_point_f,mean_humidity,mean_sea_level_pressure_inches,mean_visibility_miles,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,events,wind_dir_degrees
0,504737,autumn,60,71,11,5,Customer,65.0,54.0,64.0,29.94,10.0,6.0,28.0,0,Clear,285.0
1,505036,autumn,60,61,17,5,Subscriber,65.0,54.0,64.0,29.94,10.0,6.0,28.0,0,Clear,285.0


In [6]:
train['mean_temperature_f'] = pd.qcut(train['mean_temperature_f'], 5)
train[['mean_temperature_f', 'duration']].groupby(['mean_temperature_f'], as_index=False).mean().sort_values(by='mean_temperature_f', ascending=True)

,mean_temperature_f,duration
0,"[38, 56]",1009.471835
1,"(56, 59]",984.504272
2,"(59, 63]",1245.619184
3,"(63, 67]",1179.927536
4,"(67, 84]",1129.538014


In [7]:
train['mean_dew_point_f'] = pd.qcut(train['mean_dew_point_f'], 4)
train[['mean_dew_point_f', 'duration']].groupby(['mean_dew_point_f'], as_index=False).mean().sort_values(by='mean_dew_point_f', ascending=True)

,mean_dew_point_f,duration
0,"[13, 46]",1042.688781
1,"(46, 50]",1051.163972
2,"(50, 55]",1076.855895
3,"(55, 65]",1318.130032


In [8]:
train['mean_humidity'] = pd.qcut(train['mean_humidity'], 4)
train[['mean_humidity', 'duration']].groupby(['mean_humidity'], as_index=False).mean().sort_values(by='mean_humidity', ascending=True)

,mean_humidity,duration
0,"[24, 64]",1098.271584
1,"(64, 69]",1121.003302
2,"(69, 73]",1085.310515
3,"(73, 96]",1133.534873


In [9]:
train['mean_sea_level_pressure_inches'] = pd.qcut(train['mean_sea_level_pressure_inches'], 4)
train[['mean_sea_level_pressure_inches', 'duration']].groupby(['mean_sea_level_pressure_inches'], as_index=False).mean().sort_values(by='mean_sea_level_pressure_inches', ascending=True)


,mean_sea_level_pressure_inches,duration
0,"[29.43, 29.92]",1087.670092
1,"(29.92, 30]",1158.508659
2,"(30, 30.1]",1061.014187
3,"(30.1, 30.41]",1146.956470


In [10]:
train['mean_wind_speed_mph'] = pd.qcut(train['mean_wind_speed_mph'], 4)
train[['mean_wind_speed_mph', 'duration']].groupby(['mean_wind_speed_mph'], as_index=False).mean().sort_values(by='mean_wind_speed_mph', ascending=True)

,mean_wind_speed_mph,duration
0,"[0, 5]",1110.747702
1,"(5, 8]",1220.228930
2,"(8, 10]",998.600309
3,"(10, 22]",1097.121257


In [11]:
train['max_gust_speed_mph'] = pd.qcut(train['max_gust_speed_mph'], 6)
train[['max_gust_speed_mph', 'duration']].groupby(['max_gust_speed_mph'], as_index=False).mean().sort_values(by='max_gust_speed_mph', ascending=True)

,max_gust_speed_mph,duration
0,"[6, 17]",1047.843041
1,"(17, 21]",1265.753621
2,"(21, 23]",1053.089657
3,"(23, 25]",1086.799882
4,"(25, 30]",1095.189937
5,"(30, 114]",1013.928496


In [12]:
train['wind_dir_degrees'] = pd.qcut(train['wind_dir_degrees'], 6)
train[['wind_dir_degrees', 'duration']].groupby(['wind_dir_degrees'], as_index=False).mean().sort_values(by='wind_dir_degrees', ascending=True)

,wind_dir_degrees,duration
0,"[0, 232]",1219.980691
1,"(232, 265]",964.206437
2,"(265, 277]",1014.358952
3,"(277, 291]",1058.948644
4,"(291, 305]",1115.633949
5,"(305, 2772]",1316.708177


In [13]:
def pct_rank_qcut(series, n):
    edges = pd.Series([float(i) / n for i in range(n + 1)])
    f = lambda x: (edges >= x).argmax()
    return series.rank(pct=1).apply(f)

train['mean_visibility_miles'] = train['mean_visibility_miles'].astype(float)
train['mean_visibility_miles'] = pct_rank_qcut(train.mean_visibility_miles, 5)

train[['mean_visibility_miles', 'duration']].groupby(['mean_visibility_miles'], as_index=False).mean().sort_values(by='mean_visibility_miles', ascending=True)

,mean_visibility_miles,duration
0,0,4314.302521
1,1,970.738084
2,4,1137.127259
3,5,4002.273876


In [14]:
    train_filtered['mean_temperature_f'] = pd.qcut(train_filtered['mean_temperature_f'], 5)
    train_filtered['mean_dew_point_f'] = pd.qcut(train_filtered['mean_dew_point_f'], 4)
    train_filtered['mean_humidity'] = pd.qcut(train_filtered['mean_humidity'], 4)
    train_filtered['mean_sea_level_pressure_inches'] = pd.qcut(train_filtered['mean_sea_level_pressure_inches'], 4)
    train_filtered['mean_wind_speed_mph'] = pd.qcut(train_filtered['mean_wind_speed_mph'], 4)
    train_filtered['max_gust_speed_mph'] = pd.qcut(train_filtered['max_gust_speed_mph'], 6)
    train_filtered['wind_dir_degrees'] = pd.qcut(train_filtered['wind_dir_degrees'], 6)
    '''CHEAQUEAR ESTO'''
    train_filtered['mean_visibility_miles'] = train_filtered['mean_visibility_miles'].astype(float)
    train_filtered['mean_visibility_miles'] = pct_rank_qcut(train_filtered.mean_visibility_miles, 5)

In [15]:
    test['mean_temperature_f'] = pd.qcut(test['mean_temperature_f'], 5)
    test['mean_dew_point_f'] = pd.qcut(test['mean_dew_point_f'], 4)
    test['mean_humidity'] = pd.qcut(test['mean_humidity'], 4)
    test['mean_sea_level_pressure_inches'] = pd.qcut(test['mean_sea_level_pressure_inches'], 4)
    test['mean_wind_speed_mph'] = pd.qcut(test['mean_wind_speed_mph'], 4)
    test['max_gust_speed_mph'] = pd.qcut(test['max_gust_speed_mph'], 6)
    test['wind_dir_degrees'] = pd.qcut(test['wind_dir_degrees'], 6)
    '''CHEAQUEAR ESTO'''
    test['mean_visibility_miles'] = test['mean_visibility_miles'].astype(float)
    test['mean_visibility_miles'] = pct_rank_qcut(test.mean_visibility_miles, 5)    

In [26]:
    from sklearn import preprocessing
    features = ['season', 'subscription_type', 'mean_temperature_f', 'mean_dew_point_f',  'mean_humidity', 'mean_sea_level_pressure_inches','mean_visibility_miles', 'mean_wind_speed', 'max_gust_speed', 'precipitation_inches', 'events',  'wind_dir_degrees']
    
    for feature in features:
        le = preprocessing.LabelEncoder()
        le = le.fit(df_combined[feature])
        train[feature] = le.transform(train[feature])
        test[feature] = le.transform(test[feature])

/usr/local/lib/python2.7/dist-packages/numpy/lib/arraysetops.py:459: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)


ValueError: y contains new labels: [0 1 2 3]